# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker
!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from ast import literal_eval
from collections import Counter
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
)
from transformers.optimization import (
    Adafactor,
    get_linear_schedule_with_warmup,
)

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

/home/selim/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
DATA_PATH = os.path.join('..', '..', '..', "data", "secondary_tags", "severity_tags_final.csv")

df = pd.read_csv(DATA_PATH, index_col=0, lineterminator='\n')

train_df, val_df = train_test_split(df, test_size=0.2)

In [7]:
train_df.head(2)

,entry_id,excerpt,severity
536756,292319,Enfrentamiento armado en Murindó (Antioquia) p...,['Critical']
121539,59628,Cases of pregnancies among girls as young as 1...,['Major']


## Sagemaker Prep

### Session

Configure SageMaker

In [8]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

/home/selim/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Bucket upload

You need to upload data to an S3 bucket. 




In [9]:
sample = False  # To make the computations faster, sample = True.

if sample:
    train_df = train_df.sample(n=1000)
    val_df = val_df.sample(n=1000)
    
job_name = f"pytorch-{formatted_time()}-subpillars-model-test-mlflow"  # cannot be changed
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


train_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
val_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [10]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [11]:
from sagemaker.pytorch import PyTorch


hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'max_len': 256,
    'epochs': 3,
    'model_name': 'microsoft/xtremedistil-l6-h384-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h384-uncased',
    'dropout_rate': 0.3,
    'pred_threshold':0.4,
    'output_length': 384,
    'learning_rate': 7e-5,
    'experiment_name': "pl-severity",
    'training_column':'severity',
    'multiclass_bool':False,
    'train_with_whole_dataset':False
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type='ml.p2.xlarge',
    instance_count=1,
    role=role,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [12]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [13]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-08-19 14:20:55 Starting - Starting the training job...
2021-08-19 14:21:19 Starting - Launching requested ML instancesProfilerReport-1629382852: InProgress
......
2021-08-19 14:22:19 Starting - Preparing the instances for training.........
2021-08-19 14:24:00 Downloading - Downloading input data...
2021-08-19 14:24:39 Training - Downloading the training image.........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-08-19 14:29:01,844 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-08-19 14:29:01,876 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-08-19 14:29:01,885 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-08-19 14:29:02,401 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt



  Created wheel for nltk: filename=nltk-3.2.5-py3-none-any.whl size=1392141 sha256=4a6da33ff70335e6b9757a5810202522bf41501375ed5750528fec600bb3cca0
  Stored in directory: /root/.cache/pip/wheels/f2/7f/71/cb36468789a03b5e2908281c8e1ce093e6860258b6b61677d8
  Created wheel for sagemaker: filename=sagemaker-2.49.1-py2.py3-none-any.whl size=591917 sha256=170457950b6e8c8810d40eb10831de787f77cf9eaaaf288e137292105a04cc0d
  Stored in directory: /root/.cache/pip/wheels/4c/af/ea/8ff5943a87155df5b184e54474fbf2b59b75e5c172854643c6
  Created wheel for aiobotocore: filename=aiobotocore-1.3.3-py3-none-any.whl size=48336 sha256=7ce38d00b8e043122ec9017416598aac4c97d22f3dfd36b428b7d4fcb1550458
  Stored in directory: /root/.cache/pip/wheels/b8/12/24/4cc15fc7666d3811f9627ea675972ff0c6d3d56bd5b53aa8ab
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=e9c0b4a090dc3e2f8acc22c93806de1d04ffbc6f1cd6979e2fbee2e2223e4a9c
  Stored in directory: /root/.cache/pip/wheels/e9/7b

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=3a4c970b40dd7bf2916b8f3681f58945474cb56d2cf72b59c924c5faaacf504b
  Stored in directory: /root/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc
  Created wheel for wrapt: filename=wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl size=69750 sha256=25bca8bd78df9bda07657ba0b637c58a8741f2b6d702a325ccebe70395cbfcb5
  Stored in directory: /root/.cache/pip/wheels/32/42/7f/23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63
  Created wheel for prometheus-flask-exporter: filename=prometheus_flask_exporter-0.18.2-py3-none-any.whl size=17398 sha256=809f5ac3b2a6aec394e29cd2411d15e2cbf10969caae921e42179dd35e04dc9e
  Stored in directory: /root/.cache/pip/wheels/15/77/e8/3ca90b66243b0b58d5a5323a3da02cc8c5daf1de7a65141701
Successfully built nltk sagemaker aiobotocore alembic databricks-cli idna-ssl termcolor wrapt prometheus-flask-exporter
  Attempting uninstall: six
    Found existing installa

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
INFO: 'pl-severity' does not exist. Creating a new experiment
#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s][2021-08-19 14:31:52.970 algo-1:91 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-08-19 14:31:53.010 algo-1:91 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-08-19 14:31:53.011 algo-1:91 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-08-19 14:31:53.011 algo-1:91 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tens

#015                                                              #015#015Training: 0it [00:00, ?it/s]#015Training:   0%|          | 0/1355 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/1355 [00:00<?, ?it/s] #015Epoch 0:   2%|▏         | 30/1355 [00:09<07:01,  3.15it/s]#015Epoch 0:   2%|▏         | 30/1355 [00:09<07:01,  3.15it/s, loss=3.1, v_num=0, val_f1_epoch=0.167, val_loss_epoch=0.704, train_f1=0.589]#015Epoch 0:   4%|▍         | 60/1355 [00:18<06:44,  3.20it/s, loss=3.1, v_num=0, val_f1_epoch=0.167, val_loss_epoch=0.704, train_f1=0.589]#015Epoch 0:   4%|▍         | 60/1355 [00:18<06:44,  3.20it/s, loss=1.03, v_num=0, val_f1_epoch=0.167, val_loss_epoch=0.704, train_f1=0.627]#015Epoch 0:   7%|▋         | 90/1355 [00:27<06:31,  3.23it/s, loss=1.03, v_num=0, val_f1_epoch=0.167, val_loss_epoch=0.704, train_f1=0.627]#015Epoch 0:   7%|▋         | 90/1355 [00:27<06:31,  3.23it/s, loss=1.93, v_num=0, val_f1_epoch=0.167, val_loss_epoch=0.704, train_f1=0.621]#015Epoch 0:   9%|▉         | 

#015Validating: 100%|██████████| 124/124 [00:26<00:00,  5.04it/s]#033[A#015Epoch 0: 100%|██████████| 1355/1355 [06:48<00:00,  3.32it/s, loss=0.881, v_num=0, val_f1_epoch=0.779, val_loss_epoch=0.293, train_f1=0.652, val_f1_step=0.844, val_loss_step=0.153]
#015                                                             #033[A#015Epoch 0:   0%|          | 0/1355 [00:00<?, ?it/s, loss=0.881, v_num=0, val_f1_epoch=0.779, val_loss_epoch=0.293, train_f1=0.652, val_f1_step=0.844, val_loss_step=0.153]           #015Epoch 1:   0%|          | 0/1355 [00:00<?, ?it/s, loss=0.881, v_num=0, val_f1_epoch=0.779, val_loss_epoch=0.293, train_f1=0.652, val_f1_step=0.844, val_loss_step=0.153]#015Epoch 1:   2%|▏         | 30/1355 [00:09<07:00,  3.15it/s, loss=0.881, v_num=0, val_f1_epoch=0.779, val_loss_epoch=0.293, train_f1=0.652, val_f1_step=0.844, val_loss_step=0.153]#015Epoch 1:   2%|▏         | 30/1355 [00:09<07:00,  3.15it/s, loss=1.25, v_num=0, val_f1_epoch=0.779, val_loss_epoch=0.293, train_f1=0.76

#015Validating: 100%|██████████| 124/124 [00:26<00:00,  5.02it/s]#033[A#015Epoch 1: 100%|██████████| 1355/1355 [06:49<00:00,  3.31it/s, loss=0.79, v_num=0, val_f1_epoch=0.812, val_loss_epoch=0.261, train_f1=0.868, val_f1_step=0.844, val_loss_step=0.149] 
#015                                                             #033[A#015Epoch 1: 100%|██████████| 1355/1355 [07:00<00:00,  3.23it/s, loss=0.79, v_num=0, val_f1_epoch=0.812, val_loss_epoch=0.261, train_f1=0.868, val_f1_step=0.844, val_loss_step=0.149]#015Epoch 1:   0%|          | 0/1355 [00:00<?, ?it/s, loss=0.79, v_num=0, val_f1_epoch=0.812, val_loss_epoch=0.261, train_f1=0.868, val_f1_step=0.844, val_loss_step=0.149]           #015Epoch 2:   0%|          | 0/1355 [00:00<?, ?it/s, loss=0.79, v_num=0, val_f1_epoch=0.812, val_loss_epoch=0.261, train_f1=0.868, val_f1_step=0.844, val_loss_step=0.149]#015Epoch 2:   2%|▏         | 30/1355 [00:09<07:00,  3.15it/s, loss=0.79, v_num=0, val_f1_epoch=0.812, val_loss_epoch=0.261, train_f1=0.868

#015Validating: 100%|██████████| 124/124 [00:26<00:00,  5.01it/s]#033[A#015Epoch 2: 100%|██████████| 1355/1355 [06:49<00:00,  3.31it/s, loss=1.03, v_num=0, val_f1_epoch=0.827, val_loss_epoch=0.247, train_f1=0.766, val_f1_step=0.844, val_loss_step=0.183]
#015                                                             #033[A#015Epoch 2: 100%|██████████| 1355/1355 [06:50<00:00,  3.30it/s, loss=1.03, v_num=0, val_f1_epoch=0.827, val_loss_epoch=0.247, train_f1=0.766, val_f1_step=0.844, val_loss_step=0.183]
(7771, 5)
(7771, 5)
2021-08-19 14:30:30.230416: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
INFO:root:reading, preprocessing data
2021/08/19 14:30:33 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
INFO:filelock:Lock 140251513458360 acquired on /ro


2021-08-19 14:54:25 Uploading - Uploading generated training model
2021-08-19 14:55:00 Failed - Training job failed


UnexpectedStatusException: Error for Training job pytorch-2021-08-19-16-19-39-920-subpillars-model-test-mlflow: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 train_mlflow.py --dropout_rate 0.3 --epochs 3 --experiment_name pl-severity --learning_rate 7e-05 --max_len 256 --model_name microsoft/xtremedistil-l6-h384-uncased --multiclass_bool False --output_length 384 --pred_threshold 0.4 --tokenizer_name microsoft/xtremedistil-l6-h384-uncased --tracking_uri http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/ --train_with_whole_dataset False --training_column severity"
2021-08-19 14:30:30.230416: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
INFO:root:reading, preprocessing data
2021/08/19 14:30:33 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pyto